In [8]:
#this is to import dataframe from other notebooks
#before importing we need to have converted .ipynb to .py files using the command - 
#!jupyter nbconvert --to script Buildings_Exploratory.ipynb  
#at the end of each .ipynb file

import Agent_Data_First_Look as agents
import Buildings_Exploratory as build

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
exits = build.df
agents = agents.df

In [26]:
agents.head()

,sec,agent_id,agent_type,has_luggage,is_disabled,X,Y,Z,velocity,queue_id,lookUp_X,lookUp_Y
0,70201,0,4,0,0,105.955,20.712,7.0,0.586,-1,105.500,22.130
1,70201,8,4,0,0,82.995,14.989,1.0,0.720,-1,83.495,18.265
2,70201,16,1,0,0,49.066,-72.638,1.0,0.737,-1,49.822,-72.954
3,70201,24,1,0,0,111.617,28.309,10.0,2.944,-1,108.060,30.175
4,70201,32,1,0,0,108.920,34.867,4.0,1.386,-1,103.980,32.855


In [105]:
exits.head()

,name,no_of_triangles,triangles,x,y,z
0,entry__exit_4_58,6,"[(100.6, -17.01, 6.0), (100.6, -11.75, 6.0), (...","[100.6, 100.6, 92.88, 100.6, 92.88, 92.88]","[-17.01, -11.75, -11.75, -17.01, -11.75, -17.01]","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0]"
2,entry_exit_7_455,6,"[(120.12, 60.33, 6.0), (122.93, 64.5, 6.0), (1...","[120.12, 122.93, 120.16, 120.12, 123.0, 122.93]","[60.33, 64.5, 64.5, 60.33, 60.28, 64.5]","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0]"
4,entry_exit_stairs_544,6,"[(46.24, 9.3, 6.0), (53.04, 9.3, 6.0), (53.04,...","[46.24, 53.04, 53.04, 46.24, 53.04, 46.08]","[9.3, 9.3, 17.1, 9.3, 17.1, 16.96]","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0]"
6,entry_exit_591,9,"[(107.77, 10.18, 6.0), (107.42, 16.32, 6.0), (...","[107.77, 107.42, 103.53, 103.54, 107.77, 107.7...","[10.18, 16.32, 16.7, 9.3, 8.9, 10.18, 9.3, 10....","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]"
8,exit_entry_1_1002,6,"[(32.3, -20.64, 6.0), (36.34, -11.79, 6.0), (3...","[32.3, 36.34, 32.3, 32.3, 36.36, 36.34]","[-20.64, -11.79, -11.75, -20.64, -20.75, -11.79]","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0]"


In [103]:
#taking only the minimum time, that is the time each agent entered the building
df = agents.sort_values("sec").groupby("agent_id", as_index=False).first() #1063 rows in df 
df.head()

,agent_id,sec,agent_type,has_luggage,is_disabled,X,Y,Z,velocity,queue_id,lookUp_X,lookUp_Y
0,0,70201,4,0,0,105.955,20.712,7.0,0.586,-1,105.500,22.130
1,8,70201,4,0,0,82.995,14.989,1.0,0.720,-1,83.495,18.265
2,16,70201,1,0,0,49.066,-72.638,1.0,0.737,-1,49.822,-72.954
3,24,70201,1,0,0,111.617,28.309,10.0,2.944,-1,108.060,30.175
4,32,70201,1,0,0,108.920,34.867,4.0,1.386,-1,103.980,32.855


In [88]:
#Question-1 : Through which entrance did each of the agents enter and when??
#Approach - take the minimum time for each agent and the corresponding X,Y,Z coordinate
#Take the euclidean distance of each agent's starting point with each triangle in the each entrance
#there are about 450 triangles in total - so pretty inefficient - but more accurate - we can cross check with mean method later
#take the closest entrance

from scipy.spatial import distance
df_list = []
dist = 100
row_num = 100
#calculating euclidean distance
for i in df.iterrows(): #each agent going into the loop
    starting_point = (i[1]['X'],i[1]['Y'],i[1]['Z'])
    for j in exits.iterrows(): #each exit going into the loop
        for k in j[1][2]: #each triangle going into the loop
            d = distance.euclidean(k,starting_point) #distance from each triangle to starting point of agent
            if d < dist:
                dist = d
                entrance = j[1]['name']
                triangle = k
                
    data = {'agent_id' : i[1]['agent_id'],
            'entry_time' : i[1]['sec'],
            'agent_type' : i[1]['agent_type'],
            'has_luggage' :  i[1]['has_luggage'],
            'is_disabled' : i[1]['is_disabled'],
            'starting_point' : starting_point,
            'entrance' : entrance,
            'dist' : dist,
            'triangle' : triangle}
    df_list.append(data)

df = pd.DataFrame(df_list)
df.head()
    

,agent_id,agent_type,dist,entrance,entry_time,has_luggage,is_disabled,starting_point,triangle
0,0.0,4.0,4.736654,entry_exit_591,70201.0,0.0,0.0,"(105.955, 20.712, 7.0)","(107.42, 16.32, 6.0)"
1,8.0,4.0,4.736654,entry_exit_591,70201.0,0.0,0.0,"(82.995, 14.989, 1.0)","(107.42, 16.32, 6.0)"
2,16.0,1.0,4.736654,entry_exit_591,70201.0,0.0,0.0,"(49.066, -72.638, 1.0)","(107.42, 16.32, 6.0)"
3,24.0,1.0,4.736654,entry_exit_591,70201.0,0.0,0.0,"(111.617, 28.309, 10.0)","(107.42, 16.32, 6.0)"
4,32.0,1.0,3.945350,Subway_Entrance_2_12714,70201.0,0.0,0.0,"(108.92, 34.867, 4.0)","(111.65, 32.2, 3.0)"


In [100]:
counts = df.groupby('entrance').size()
counts
#distribution of entrances among agents
#Subway_Entrance_2_12714    and SouthEntrance1_13925    are the MOST popular

entrance
NorthExit_14267             11
SouthEntrance1_13925       476
Subway_Entrance_2_12714    572
entry_exit_591               4
dtype: int64

In [101]:
df.dist.describe()
#distribution of distance

count    1063.000000
mean        1.219535
std         0.299311
min         1.121257
25%         1.121257
50%         1.212329
75%         1.230585
max         4.736654
Name: dist, dtype: float64

In [ ]:
#NOW we can try the same by calculating the distance from (mean(x),mean(y) and mean(z)) and check if the results match